- 2D Search -  Freq [Liver, Colon, Pancreas, Lung]

In [ ]:
import faiss 
import pandas as pd
import torch
from tqdm.notebook import tqdm
import glob 
import pickle
import numpy as np
from random import sample
import os
import open_clip
import csv
from sklearn.metrics import average_precision_score
import math
####################################################
## Select one of the organs: liver |colon | pancreas | lung
organ="liver"
####################################################

## Methods:
method1 = 'biomedclip_caption-based' # caption-based search method
method2 = 'biomedclip_caption+slice' # hybrid method combining the caption- and slice-based searches

## Quantification Files
quantification_dir = "data/msd_final/quantification_file"
quantification_csv = quantification_dir + "/msd_" + organ + "_metrics_3D_subject_including_normal_tissue.csv"

data_splits_folder = "data/msd_train_test_split"
if (organ == "lung"):
    data_splits_folder = "data/msd_train_test_split/lung"

all_embs_folder = "data/msd_embeddings/biomedclip_2d"

# Image Captions
captionsFile="msd_gpt4_captions/gpt4_captions_msd_"+organ+".csv" 

test_set_csv =  data_splits_folder + "/" + organ + "/" + organ + "_full_test_split.csv"
## Output files 
results_dir = "BiomedCLIP/MSD-Final/results/results_" + organ
results_eval_metrics = results_dir + "/" + organ + "eval_metrics.csv"

## 2D Index for the slice-based and caption-based searches
ids_organ_path = "BiomedCLIP/MSD-Final/2d-index/" + organ + "-train-2d-index/faiss_ids_embd_train.tsv"
index_organ_path = "BiomedCLIP/MSD-Final/2d-index/" + organ + "-train-2d-index/faiss_index_embd_train.index"


In [ ]:
class FAISSRetrievalEngine:
    def __init__(self, ids_path, index_path):
        self.ids_path = ids_path
        self.ids = pd.read_csv(self.ids_path, sep="\t")
        self.ids = {row["ids_int"]: row["ids_str"] for i, row in self.ids.iterrows()}

        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.index_path = index_path
        self.index = faiss.read_index(self.index_path)
        if self.device == "cuda":
            res = faiss.StandardGpuResources()
            self.index = faiss.index_cpu_to_gpu(res, 0, self.index)
    
    def _split_array_into_batches(self, arr, batch_size):
        num_rows = arr.shape[0]
        num_batches = (num_rows + batch_size - 1) // batch_size  
        
        # Create a list of batches
        batches = []
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, num_rows)
            batch = arr[start_idx:end_idx, :]
            batches.append(batch)

        return batches

    def get_str_ids_from_I(self, I):
        mapping_fn = lambda x: self.ids[x]
        return np.vectorize(mapping_fn)(I)

    def retrieve_images(self, xq, k=5):
        batches = self._split_array_into_batches(xq, 16)
        output_D, output_I = [], []
        for batch in tqdm(batches, total=len(batches)):
            D, I = self.index.search(batch, k)
            I = self.get_str_ids_from_I(I)
            output_D.append(D)
            output_I.append(I)
        output_D = np.concatenate(output_D, axis=0)
        output_I = np.concatenate(output_I, axis=0)
        return output_D, output_I

class BiomedCLIPTextEmbeddingGenerator:
    def __init__(self):
        self.model, _, _ = open_clip.create_model_and_transforms(
            'hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224'
            )
        self.tokenizer = open_clip.get_tokenizer(
            'hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224'
            )
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.model.to(self.device)
    

    def __call__(self, phrases):
        if not isinstance(phrases, list):
            phrases = [phrases]
        
        texts = self.tokenizer(phrases, context_length=256).to(self.device)
        with torch.no_grad():
            _, text_features, _ = self.model(None, texts)
        
        return text_features.cpu().detach().numpy()

class Text2ImageRetrieval: 
    def __init__(self, ids_path, index_path):        
        self.index = FAISSRetrievalEngine(ids_path, index_path)
        print(" (1/2) index: done")
        self.model = BiomedCLIPTextEmbeddingGenerator()
        print("(2/2) model: done")
    
    def __call__(self, phrases, k=4):
        embd = self.model(phrases)
        embd_norm = embd / np.linalg.norm(embd, axis=1)[:, np.newaxis]
        D, I = self.index.retrieve_images(embd_norm, 20) ## 20 slices 
        return D, I

#######################################
## Image2ImageRetrieval
index = FAISSRetrievalEngine(
    ids_path=ids_organ_path,
    index_path=index_organ_path
)

## Text2ImageRetrieval
ret_engine = Text2ImageRetrieval(
    ids_path=ids_organ_path,
    index_path=index_organ_path
)
#######################################

**Running the Image Search for all test queries  
**

In [ ]:
from collections import Counter    
  
def sort_by_frequency(lst):  
   
    freq_dict = Counter(lst)   
    return sorted(lst, key=lambda x: (-freq_dict[x], x))  


def sort_by_sum_scores(results):  
   
    sorted_ids = sorted(results.keys(), key=lambda x: -np.array(results[x]).sum())
    return sorted_ids 

def sort_by_max_scores(results):  
   
    sorted_ids = sorted(results.keys(), key=lambda x: -np.array(results[x]).max())
    return sorted_ids 
    
def get_3D_results(slice_id_arr):
    """Baseline to get 3D img results from 2D slice results.
    Uses 2D slice count of each 3D image in the result list.

    input:
     [
        [slice_1_1, slice 1_2, ..], #results for query slice 1
        [slice_2_1, slice 2_2, ..], #results for query slice 2
        ...
     ]

     output:
     [ img_id_1, img_id_2, .., img_id_n] # initial 3D images ranked by slice count

    """
    results = []

    for i, slice_ids in enumerate(slice_id_arr):
        for slice_id in slice_ids:
            slice_str = str(slice_id)
            img_id = slice_str[:slice_str.rindex('_')]
            results.append(img_id)

    ## Select the aggregation method: max | sum | freq 
    #results = sort_by_max_scores(results)
    #results = sort_by_sum_scores(results) 
    results = sort_by_frequency(results)
    result_set = set()
    final_results = []

    for r in results:
        if r not in result_set:
            final_results.append(r)
            result_set.add(r)
    return final_results

In [ ]:
### SLICE-BASED SEARCH ###
search_results1 = {'query_img': [], 'results': []}
test_csv = pd.read_csv(test_set_csv)
for img in test_csv['testing']: 
    query_img_id = img[:img.index('.')]
    test_volume_emb = os.path.join(all_embs_folder, query_img_id + '.pkl')
    search_results1['query_img'].append(query_img_id)
    with open(test_volume_emb, "rb") as f:
        embd = pickle.load(f)
    D, I = index.retrieve_images(embd, 20) ## retrieving 20 slices for eval
    results = get_3D_results(I)
    search_results1['results'].append(' '.join(results))
df1 = pd.DataFrame(search_results1)

In [ ]:
### TEXT-BASED SEARCH ###
search_results2 = {'query_img': [], 'results': []}
test_csv = pd.read_csv(test_set_csv)
captions_df = pd.read_csv(captionsFile) 
for img in test_csv['testing']: 
    query_img_id = img[:img.index('.')]
    search_results2['query_img'].append(query_img_id)
    caption=captions_df.loc[captions_df['filename']==query_img_id]['caption']
    caption=str(caption).replace('"', "")

    D, I = ret_engine(caption, k=20) ## retrieving 20 slices for eval
    results = get_3D_results(I)
    search_results2['results'].append(' '.join(results))
df2 = pd.DataFrame(search_results2)
df2.to_csv(results_dir + '/' + method1 + '_results.csv', index=False)

In [ ]:
aggregated_df = pd.read_csv(quantification_csv) 
aggregated_df['name'] = aggregated_df['name'].replace('.nii.gz', '')
for i in range(len(aggregated_df['name'])):
    filename = aggregated_df['name'][i]
    aggregated_df['name'][i] = filename[:filename.index('.')] 
aggregated_df.head()

In [ ]:
##########################################################################################
            # Evaluation of Caption-based Search 
##########################################################################################
# Eval Top-k
tumor_stage_eval  = {'query': [] } 
tumor_flag_eval = {'query': [] }

for i in range(10):
    tumor_stage_eval[f"Top {i+1}"] = []
    tumor_flag_eval[f"Top {i+1}"] = []

for query, res in zip(search_results2['query_img'], search_results2['results']):
    query_metrics = aggregated_df.loc[aggregated_df['name'] == query].iloc[0]
    
    tumor_stage_eval['query'].append(query)
    tumor_flag_eval['query'].append(query)

    tumor_flag = organ + "_cancer_flag" 
    top_n = res.split(' ')
    for k, top_k in enumerate(top_n):
        if k > 9:
            break

        res_metrics = aggregated_df.loc[aggregated_df['name'] == top_k].iloc[0]
        # tumor stage
        tumorStage=0
        if(query_metrics['cancer_stage']-res_metrics['cancer_stage'] == 0):
            tumorStage=1 

        tumor_stage_eval[f"Top {k+1}"].append(tumorStage)
        tumor_flag_eval[f"Top {k+1}"].append(1-abs(query_metrics[tumor_flag]-res_metrics[tumor_flag]))
        
    if len(top_n) < 10:
        for k in range(len(top_n), 10):
            tumor_stage_eval[f"Top {k+1}"].append('NA')
            tumor_flag_eval[f"Top {k+1}"].append('NA')

tumor_stage_df = pd.DataFrame(tumor_stage_eval)
tumor_stage_df.to_csv(results_dir + '/' + method1 + '_tumor_stage_eval.csv', index=False) 
tumor_flag_df = pd.DataFrame(tumor_flag_eval)
tumor_flag_df.to_csv(results_dir + '/' + method1 + '_tumor_flag_eval.csv', index=False) 

##########################################################################################    
print("### Results of Caption-based Search")
## Tumor Flag
print("# Tumor Flag: ")
tumor_flag_df['RowMean'] = tumor_flag_df.iloc[:, 1:4].mean(axis=1)
overall_mean = tumor_flag_df['RowMean'].mean()
print("tumor_flag_p@3: ", '{:.4f}'.format(overall_mean))

tumor_flag_df['RowMean'] = tumor_flag_df.iloc[:, 1:6].mean(axis=1)
overall_mean = tumor_flag_df['RowMean'].mean()
print("tumor_flag_p@5: ", '{:.4f}'.format(overall_mean))

tumor_flag_df['RowMean'] = tumor_flag_df.iloc[:, 1:].mean(axis=1)
overall_mean = tumor_flag_df['RowMean'].mean()
print("tumor_flag_p@10: ", '{:.4f}'.format(overall_mean))

## Tumor Stage
print("# Tumor Stage: ")
tumor_stage_df['RowMean'] = tumor_stage_df.iloc[:, 1:4].mean(axis=1)
overall_mean = tumor_stage_df['RowMean'].mean()
print("tumor_stage_p@3: ", '{:.4f}'.format(overall_mean))

tumor_stage_df['RowMean'] = tumor_stage_df.iloc[:, 1:6].mean(axis=1)
overall_mean = tumor_stage_df['RowMean'].mean()
print("tumor_stage_p@5: ", '{:.4f}'.format(overall_mean))

tumor_stage_df['RowMean'] = tumor_stage_df.iloc[:, 1:].mean(axis=1)
overall_mean = tumor_stage_df['RowMean'].mean()
print("tumor_stage_p@10: ", '{:.4f}'.format(overall_mean))

## Method 2 - Average Precision

In [ ]:
ref_tumor_flag = []
all_res_tumor_flag = []
ref_tumor_stage = []
all_res_tumor_stage = []

search_results = search_results2

for query, res in zip(search_results['query_img'], search_results['results']):
    query_metrics = aggregated_df.loc[aggregated_df['name'] == query].iloc[0]
    tumor_flag = organ + "_cancer_flag" 
    ref_tumor_flag.append(query_metrics[tumor_flag])
    ref_tumor_stage.append(query_metrics['cancer_stage'])
    top_n = res.split(' ')
    res_tumor_flag = []
    res_tumor_stage = []
    for k, top_k in enumerate(top_n):
        if k > 9:
            break
        res_metrics = aggregated_df.loc[aggregated_df['name'] == top_k].iloc[0]
        res_tumor_flag.append(res_metrics[tumor_flag])
        res_tumor_stage.append(res_metrics['cancer_stage'])
    all_res_tumor_flag.append(res_tumor_flag)
    all_res_tumor_stage.append(res_tumor_stage)


## Adaptation for Average Precision (AP) computation
## AP - Tumor Flag

for i, q in enumerate(ref_tumor_flag):
    if q == 0:
        for j, res in enumerate(all_res_tumor_flag[i]):
            if res == 0:
                all_res_tumor_flag[i][j] = 1
            else:
                all_res_tumor_flag[i][j] = 0

## AP - Tumor Stage

for i, q in enumerate(ref_tumor_stage):
    print(q)
    for j, res in enumerate(all_res_tumor_stage[i]):
        if res == q:
            all_res_tumor_stage[i][j] = 1
        else:
            all_res_tumor_stage[i][j] = 0
            

In [ ]:
#############
## Tumor Flag
ref_tumor_flag = np.array(ref_tumor_flag)
all_res_tumor_flag = np.array(all_res_tumor_flag)

ap = 0

for i, q in enumerate(ref_tumor_flag):
    
    try:
        flag_avg_precision =  average_precision_score(all_res_tumor_flag[i], list(range(len(all_res_tumor_flag[i])))[::-1])

        if math.isnan(flag_avg_precision):
            flag_avg_precision=0

        ap = ap + flag_avg_precision
    except:
        print(all_res_tumor_flag[i])
        

print("flag_avg_precision: ", '{:.4f}'.format(ap/len(ref_tumor_flag)))


In [ ]:
#############
## Tumor Stage
ref_tumor_stage = np.array(ref_tumor_stage)
all_res_tumor_stage = np.array(all_res_tumor_stage)

ap = 0

for i, q in enumerate(ref_tumor_stage):
    
    try:
        stage_avg_precision =  average_precision_score(all_res_tumor_stage[i], list(range(len(all_res_tumor_flag[i])))[::-1])
        
        if math.isnan(stage_avg_precision):
            stage_avg_precision=0
        
        ap = ap + stage_avg_precision
    except:
        print(all_res_tumor_stage[i])

print("stage_avg_precision: ", '{:.4f}'.format(ap/len(ref_tumor_stage)))



### **Hybrid Search (captions + slices)**

In [ ]:
## Combining the search results lists
def interleave(result_list1, result_list2, k):
    interleaved_results = []

    for x, y in zip(result_list1, result_list2):
        if x not in interleaved_results:
            interleaved_results.append(x)
        
        if y not in interleaved_results:
            interleaved_results.append(y)

    if len(interleaved_results) > k:
        interleaved_results = interleaved_results[:k]

    return interleaved_results

interleaved_results = {'query_img': [], 'results': []}

i = 0
for i, query_img in enumerate(search_results1['query_img']):
    assert query_img == search_results2['query_img'][i]

    results1 = search_results1['results'][i].split(' ')
    results2 = search_results2['results'][i].split(' ')

    ir = interleave(results1, results2, 10)

    interleaved_results['query_img'].append(query_img)
    interleaved_results['results'].append(' '.join(ir))
    

interleaved_df = pd.DataFrame(interleaved_results)
interleaved_df.head()
interleaved_df.to_csv(results_dir + '/' + method2 + '_results.csv', index=False)

In [ ]:
##########################################################################################
            # Evaluation of Hybrid Search (captions+slices)
##########################################################################################
# Eval Top-k
tumor_stage_eval  = {'query': [] } 
tumor_flag_eval = {'query': [] }

for i in range(10):
    tumor_stage_eval[f"Top {i+1}"] = []
    tumor_flag_eval[f"Top {i+1}"] = []

for query, res in zip(interleaved_results['query_img'], interleaved_results['results']):
    query_metrics = aggregated_df.loc[aggregated_df['name'] == query].iloc[0]
    
    tumor_stage_eval['query'].append(query)
    tumor_flag_eval['query'].append(query)

    tumor_flag = organ + "_cancer_flag" 
    top_n = res.split(' ')
    for k, top_k in enumerate(top_n):
        if k > 9:
            break

        res_metrics = aggregated_df.loc[aggregated_df['name'] == top_k].iloc[0]
        # tumor stage
        tumorStage=0
        if(query_metrics['cancer_stage']-res_metrics['cancer_stage'] == 0):
            tumorStage=1 

        tumor_stage_eval[f"Top {k+1}"].append(tumorStage)
        tumor_flag_eval[f"Top {k+1}"].append(1-abs(query_metrics[tumor_flag]-res_metrics[tumor_flag]))
        
    if len(top_n) < 10:
        for k in range(len(top_n), 10):
            tumor_stage_eval[f"Top {k+1}"].append('NA')
            tumor_flag_eval[f"Top {k+1}"].append('NA')

tumor_stage_df = pd.DataFrame(tumor_stage_eval)
tumor_stage_df.to_csv(results_dir + '/' + method2 + '_tumor_stage_eval.csv', index=False) 
tumor_flag_df = pd.DataFrame(tumor_flag_eval)
tumor_flag_df.to_csv(results_dir + '/' + method2 + '_tumor_flag_eval.csv', index=False) 

##########################################################################################
print("### Results of Hybrid Search (Captions + slices)")    
## Tumor Flag
print("# Tumor Flag: ")
tumor_flag_df['RowMean'] = tumor_flag_df.iloc[:, 1:4].mean(axis=1)
overall_mean = tumor_flag_df['RowMean'].mean()
print("tumor_flag_p@3: ", '{:.4f}'.format(overall_mean))

tumor_flag_df['RowMean'] = tumor_flag_df.iloc[:, 1:6].mean(axis=1)
overall_mean = tumor_flag_df['RowMean'].mean()
print("tumor_flag_p@5: ", '{:.4f}'.format(overall_mean))

tumor_flag_df['RowMean'] = tumor_flag_df.iloc[:, 1:].mean(axis=1)
overall_mean = tumor_flag_df['RowMean'].mean()
print("tumor_flag_p@10: ", '{:.4f}'.format(overall_mean))

## Tumor Stage
print("# Tumor Stage: ")
tumor_stage_df['RowMean'] = tumor_stage_df.iloc[:, 1:4].mean(axis=1)
overall_mean = tumor_stage_df['RowMean'].mean()
print("tumor_stage_p@3: ", '{:.4f}'.format(overall_mean))

tumor_stage_df['RowMean'] = tumor_stage_df.iloc[:, 1:6].mean(axis=1)
overall_mean = tumor_stage_df['RowMean'].mean()
print("tumor_stage_p@5: ", '{:.4f}'.format(overall_mean))

tumor_stage_df['RowMean'] = tumor_stage_df.iloc[:, 1:].mean(axis=1)
overall_mean = tumor_stage_df['RowMean'].mean()
print("tumor_stage_p@10: ", '{:.4f}'.format(overall_mean))

## Hybrid - Average Precision

In [ ]:
ref_tumor_flag = []
all_res_tumor_flag = []
ref_tumor_stage = []
all_res_tumor_stage = []

search_results = interleaved_results

for query, res in zip(search_results['query_img'], search_results['results']):
    query_metrics = aggregated_df.loc[aggregated_df['name'] == query].iloc[0]
    tumor_flag = organ + "_cancer_flag" 
    ref_tumor_flag.append(query_metrics[tumor_flag])
    ref_tumor_stage.append(query_metrics['cancer_stage'])
    top_n = res.split(' ')
    res_tumor_flag = []
    res_tumor_stage = []
    for k, top_k in enumerate(top_n):
        if k > 9:
            break
        res_metrics = aggregated_df.loc[aggregated_df['name'] == top_k].iloc[0]
        res_tumor_flag.append(res_metrics[tumor_flag])
        res_tumor_stage.append(res_metrics['cancer_stage'])
    all_res_tumor_flag.append(res_tumor_flag)
    all_res_tumor_stage.append(res_tumor_stage)


## Adaptation for Average Precision (AP) computation
## AP - Tumor Flag

for i, q in enumerate(ref_tumor_flag):
    if q == 0:
        for j, res in enumerate(all_res_tumor_flag[i]):
            if res == 0:
                all_res_tumor_flag[i][j] = 1
            else:
                all_res_tumor_flag[i][j] = 0

## AP - Tumor Stage

for i, q in enumerate(ref_tumor_stage):
    print(q)
    for j, res in enumerate(all_res_tumor_stage[i]):
        if res == q:
            all_res_tumor_stage[i][j] = 1
        else:
            all_res_tumor_stage[i][j] = 0
            

In [ ]:
#############
## Tumor Flag
ref_tumor_flag = np.array(ref_tumor_flag)
all_res_tumor_flag = np.array(all_res_tumor_flag)

ap = 0

for i, q in enumerate(ref_tumor_flag):
    
    try:
        flag_avg_precision =  average_precision_score(all_res_tumor_flag[i], list(range(len(all_res_tumor_flag[i])))[::-1])

        if math.isnan(flag_avg_precision):
            flag_avg_precision=0

        ap = ap + flag_avg_precision
    except:
        print(all_res_tumor_flag[i])

print("flag_avg_precision: ", '{:.4f}'.format(ap/len(ref_tumor_flag)))


In [ ]:
#############
## Tumor Stage
ref_tumor_stage = np.array(ref_tumor_stage)
all_res_tumor_stage = np.array(all_res_tumor_stage)

ap = 0

for i, q in enumerate(ref_tumor_stage):
    
    try:
        stage_avg_precision =  average_precision_score(all_res_tumor_stage[i], list(range(len(all_res_tumor_flag[i])))[::-1])
        
        if math.isnan(stage_avg_precision):
            stage_avg_precision=0
        
        ap = ap + stage_avg_precision
    except:
        print(all_res_tumor_stage[i])

print("stage_avg_precision: ", '{:.4f}'.format(ap/len(ref_tumor_stage)))